In [1]:
import matplotlib.pyplot as plt
import matplotlib
# %matplotlib qt

import numpy as np
from numpy import *
from scipy.fft import fft, ifft, fftshift, ifftshift
from scipy.optimize import curve_fit

import os
import scipy.io as sio

def fft_A(A):
    fft_A = abs(fftshift(fft(A))**2)
    return fft_A

def fft_f(bandwidth=vna_bandwidth, nop=number_of_points):
    f = np.linspace(-bandwidth/2, bandwidth/2, nop)
    return f

C:\Users\Demag\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
initial_folder = r'C:\Users\Demag\PycharmProjects\ColdMeasurements\DeMag\DATA\TWPA_noise\1f noise\in_SWS_VNA'

folders = os.listdir(initial_folder)
for i in range(len(folders)):
    if ".fig" in folders[i]: pass
    else: 
        print(r"[%d]" %(i), '----------->', folders[i]) 

[0] -----------> 10_m5_flux_15_att_35.png
[1] -----------> 10_m5_flux_30_att_35.png
[2] -----------> 10_m5_flux_6_att_35.png
[3] -----------> 1f_noise_SWS att_0 flux_power -15__20-15-54.mat
[4] -----------> 1f_noise_SWS att_0 flux_power -30__20-15-54.mat
[5] -----------> 1f_noise_SWS att_0 flux_power -6__20-15-54.mat
[6] -----------> 1f_noise_SWS att_15 flux_power -15__12-40-33.mat
[7] -----------> 1f_noise_SWS att_15 flux_power -30__12-40-33.mat
[8] -----------> 1f_noise_SWS att_15 flux_power -6__12-40-33.mat
[9] -----------> 1f_noise_SWS att_35 flux_power -15__12-40-33.mat
[10] -----------> 1f_noise_SWS att_35 flux_power -30__12-40-33.mat
[11] -----------> 1f_noise_SWS att_35 flux_power -6__12-40-33.mat
[12] -----------> 25-15_flux_15_att_0.png
[13] -----------> 25-15_flux_30_att_0.png
[14] -----------> 25-15_flux_6_att_0.png
[15] -----------> m15_m5_flux_m15_att_15.png
[16] -----------> m15_m5_flux_m30_att_15.png
[17] -----------> m15_m5_flux_m6_att_15.png


In [74]:
file = folders[4]
mat = sio.loadmat(initial_folder+"\\"+file)

# flux_power = mat['flux_power'][0,0]
# flux_freq = mat['flux_freq'][0,0]
# vna_freq = mat['vna_freq'][0,0]
vna_bandwidth = mat['vna_bandwidth'][0,0]
vna_powers = mat['vna_powers'][0]
avg_total = mat['avg_total'][0,0]
number_of_points = mat['number_of_points'][0,0]

data = mat['data']

mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'flux_power', 'flux_freq', 'vna_freq', 'vna_bandwidth', 'number_of_points', 'vna_powers', 'avg_total', 'data'])

In [75]:
print(data.shape)
print(vna_powers.shape)
print(avg_total)


(6, 30, 2, 201)
(6,)
30


In [84]:
number_of_points

201

In [82]:
plt.rcParams.update({'font.size': 16})
# vna_powers, avg, mag/pha
freq = fft_f(bandwidth=vna_bandwidth, nop=number_of_points)

fig, ax = plt.subplots(1,2,figsize=(15,7), sharex=True)
ax[0].set_title('Magnitude noise')
ax[1].set_title('Phase noise')

data_vp_mag_fft = np.empty((avg_total, number_of_points))
data_vp_pha_fft = np.empty((avg_total, number_of_points))

for i in range(len(data)-4):
    p = vna_powers[i]
    data_i = data[i]
    for k in range(avg_total): 
        data_vp_mag_fft[k,:] = fft_A(data_i[k, 0,:])
        data_vp_pha_fft[k,:] = fft_A(data_i[k, 1,:])

    data_vp_mag_fft_mean = np.mean(data_vp_mag_fft, axis = 0) 
    data_vp_pha_fft_mean = np.mean(data_vp_pha_fft, axis = 0) 

    ax[0].loglog(freq, data_vp_mag_fft_mean, label = 'VNA_pow: {}'.format(p))
    ax[1].loglog(freq, data_vp_pha_fft_mean, label = 'VNA_pow: {}'.format(p))

ax[0].legend()
ax[1].legend()
ax[0].set_xlabel('Freq, Hz')
ax[1].set_xlabel('Freq, Hz')
ax[0].set_ylim(1e-7, 1e-2)
ax[1].set_ylim(1e-6, 1e-2)


from matplotlib.ticker import LogLocator, LogFormatter
# Set major and minor ticks for both axes
plt.gca().xaxis.set_major_locator(LogLocator(base=10.0, subs=np.arange(2, 10)*0.1, numticks=10))
plt.gca().yaxis.set_major_locator(LogLocator(base=10.0, subs=np.arange(2, 10)*0.1, numticks=10))

plt.gca().xaxis.set_minor_locator(LogLocator(base=10.0, subs=np.arange(1, 10)*0.1, numticks=100))
plt.gca().yaxis.set_minor_locator(LogLocator(base=10.0, subs=np.arange(1, 10)*0.1, numticks=100))

# Enable grid for both major and minor ticks


ax[1].grid()
ax[0].grid()

In [80]:
%matplotlib qt